# Intelligent Agent

An **agent** is anything that can be viewed as perceiving its environment through sensors and
acting upon that environment. Let's create an intelligent agent that can navigate through a grid world to reach a target while avoiding obstacles.

### Task 1: Environment Setup

Write a function `create_environment(num_rows, num_cols, num_obstacles)` to create a 2D list as the environment for the agent. The grid should consist of cells, some of which are obstacles (marked as "X") and one of which is the target (marked as "T"). You can represent the grid as a 2D list.
- `num_rows` represents the number of rows.
- `num_cols` represents the number of columns.
- `num_obstacles` represents the number of cells that are obstacles. The function should randomly select the row index and column index for the obstacle cells using [`numpy.random.randint`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html)
- The target cell always appears at the lower-right corner.
- The function should return the 2D list in the end.

In [28]:
import random
from IPython.display import clear_output
import time
import heapq

In [14]:
list2D = [["O", "_", "_", "_", "_"],
          ["_", "X", "_", "_", "_"],
          ["_", "_", "_", "X", "_"],
          ["_", "_", "_", "_", "_"],
          ["_", "_", "_", "_", "T"]]

In [ ]:
def create_environment(num_rows, num_cols, num_obstacles, target:list):

    # Create a 2D list with the expected size.
    # By default, each element is "◻"
    grid = [["◻" for i in range(num_cols)] for j in range(num_rows)]

    # Indicate the origin and the destination

    # Add random "⛝" to the grid.
    # The total number of "⛝" should equal to num_obstacles

    while num_obstacles > 0:
        x = random.randint(0, num_rows - 1)
        y = random.randint(0, num_cols - 1)
        if grid[x][y] != "⛝": grid[x][y] = "⛝"
        else:
            while grid[x][y] == "⛝":
                x = random.randint(0, num_rows - 1)
                y = random.randint(0, num_cols - 1)
            grid[x][y] = "⛝"
        num_obstacles -= 1

    grid[target[0]][target[1]] = 'T'

    # Invoke the print_grid() function to display the generated grid.
    for row in grid: print(' '.join(row))

    return grid

### Task 2: Agent Class

Create a class `Agent` that represents the intelligent agent.

**Attributes:**
- `self.grid`: the environment
- `self.position`: the current position of the agent, the starting position should always be the upper-left corner.
- `self.target`: the position of the target. The target's position should always be the lower-right corner.

**Methods:**
- `perceive_environment()`: This method allows the agent to perceive its adjacent cells.
- `decide_action()`: Implement a simple decision-making process for your agent to determine the next action (up, down, left, or right).
- `take_action()`: Execute the chosen action and update the agent's position on the grid.

In [59]:
class Agent:
    def __init__(self, grid, position, target):
        self.grid = grid
        self.position = tuple(position)
        self.target = tuple(target)
        self.l_pos = {}
        self.grid[self.position[0]][self.position[1]] = 'A'

    def percieve_environment(self):
        moves = {
            "north": (self.position[0] - 1, self.position[1]) if self.position[0] > 0 else None,
            "south": (self.position[0] + 1, self.position[1]) if self.position[0] < len(self.grid) - 1 else None,
            "west": (self.position[0], self.position[1] - 1) if self.position[1] > 0 else None,
            "east": (self.position[0], self.position[1] + 1) if self.position[1] < len(self.grid[0]) - 1 else None
        }
        return {d: pos for d, pos in moves.items() if pos and self.grid[pos[0]][pos[1]] in ('◻', 'T')}

    def manhattan_distance(self, position):
        return abs(position[0] - self.target[0]) + abs(position[1] - self.target[1])

    def decide_actions(self):
        moves = []
        heapq.heappush(moves, (0, self.position))
        self.l_pos = {self.position: None}
        costs = {self.position: 0}

        while moves:
            _, current = heapq.heappop(moves)

            if current == self.target:
                path = []
                while current is not None:
                    path.append(current)
                    current = self.l_pos[current]
                return path[::-1]

            for n_pos in self.percieve_environment().values():
                if n_pos not in costs and self.grid[n_pos[0]][n_pos[1]] in ('◻', 'T'):
                    n_costs = costs[current] + 1
                    costs[n_pos] = n_costs
                    priority = n_costs + self.manhattan_distance(n_pos)
                    heapq.heappush(moves, (priority, n_pos))
                    self.l_pos[n_pos] = current

        return None 

    def take_action(self):
        path = self.decide_actions()
        if not path or len(path) < 2: return 'No valid moves available. The agent is stuck!'
        n_pos = path[1]
        if self.position != self.target: self.grid[self.position[0]][self.position[1]] = '◻'
        self.position = n_pos
        self.grid[self.position[0]][self.position[1]] = 'A'

    def print_grid(self):
        """Displays the current grid state."""
        for row in self.grid:
            print(" ".join(row))
        print("\n")

In [60]:
target_x = random.randint(1, 9)
target_y = random.randint(1, 9)

start_x = random.randint(1, 9)
start_y = random.randint(1, 9)

target = [target_x, target_y]
start = [start_x, start_y]
grid = create_environment(10, 10, 20, target)

agent = Agent(grid, start, target)
moves = 0

while agent.position != agent.target:
    clear_output()
    agent.take_action()
    agent.print_grid()
    moves += 1
    time.sleep(1)

print(f'\nAgent has reached the target in {moves} moves.')

◻ ◻ ⛝ ◻ ◻ ⛝ ⛝ ◻ ◻ ◻
⛝ ◻ ◻ ◻ ◻ ⛝ ◻ ◻ ⛝ ◻
◻ ◻ ⛝ ◻ ◻ ◻ ⛝ ◻ ◻ ◻
◻ ◻ ⛝ ⛝ ◻ ◻ ◻ ◻ ◻ ◻
◻ ◻ ⛝ ◻ ⛝ ◻ ◻ ◻ ◻ ◻
◻ ◻ A ◻ ⛝ ⛝ ◻ ◻ ◻ ◻
◻ ◻ ◻ ⛝ ◻ ◻ ◻ ◻ ◻ ◻
⛝ ◻ ◻ ◻ ◻ ◻ ◻ ◻ ◻ ◻
◻ ◻ ⛝ ⛝ ◻ ◻ ◻ ◻ ◻ ⛝
◻ ⛝ ◻ ◻ ◻ ◻ ◻ ◻ ◻ T




KeyboardInterrupt: 

## Discussion:

### Search Algorithms

The algorithm we implement here is the **depth-first search** (DFS) algorithm. It is useful to search for a given node in a graph. The grid is a special case of graph where each position is a node, and all edges are either horizontal or vertical. In order to support DFS, the agent should keep two list:
- A list of all visited nodes.
- A list of all nodes leading to the current position.

Alternatively, we can implement the **breadth-first search** (BFS) algorithm to find the shortest path leading to the target. In order to implement this algorithm, the agent should analyze the entire graph first, and only take action after the shortest path is found.

### Intelligent Agents
- The **advantage** of intelligent agents is that the outcome is entirely predictable. The agent will always stick to the rules given to it.
- **Disadvantages**:
    - The agent cannot learn on its own. In order for the agent to solve the problem, the programmer must know how to solve it.
    - It can be quite challenging to explicitly describe the decision-making process.
- **Conclusion**: intelligent agents are best built for simple tasks, or where the decision rules are easy to describe.


In [ ]:
class BFSAgent:

    def __init__(self, grid):
        self.grid = grid
        self.num_rows = len(grid)
        self.num_cols = len(grid[0])
        self.position = [0, 0] # The agent always starts at the upper-left corner
        self.target = [self.num_rows-1, self.num_cols-1] # The target is always the lower-right corner
        self.path = self.analyze()
        self.step = 0
                            
    def analyze(self):
        # This method will reveal the shortest path
        queue = [self.position]
        visited_cells = [[0, 0]]
        path = {} # a dictionary to store the parent node for each visited node

        while queue:

            row, col = queue.pop(0)

            if self.grid[row][col] == "T":
                # Target found, reconstruct the path
                shortest_path = [(row, col)]
                while [row, col] != self.position: #
                    row, col = path[(row, col)]
                    shortest_path.append([row, col])
                shortest_path.reverse()
                return shortest_path

            # Explore adjacent cells
            moves = [[0, 1], [1, 0], [0, -1], [-1, 0]] # Right, Down, Left, Up
            for move in moves:
                new_row = row + move[0]
                new_col = col + move[1]
                if 0 <= new_row < self.num_rows and \
                0 <= new_col < self.num_cols and \
                self.grid[new_row][new_col] != "X" and \
                [new_row, new_col] not in visited_cells:
                    queue.append([new_row, new_col])
                    visited_cells.append([new_row, new_col])
                    path[(new_row, new_col)] = (row, col)
        return None # No path found

    def take_action(self):
        self.step += 1
        self.position = self.path[self.step]


In [ ]:
test_grid = create_environment(7, 7, 7)

agent = BFSAgent(test_grid)
print(agent.analyze())

# display_agent(agent)

for step in range(max_moves):
    agent.take_action()
    display_agent(agent)
    if agent.position[0] == agent.target[0] and \
       agent.position[1] == agent.target[1]:
        print("Target is reached.")
        break

None


NameError: name 'max_moves' is not defined